#### Sample code to Use

In [12]:
import os
import json
import pandas as pd
from glob import glob
import numpy

data = []

total_exmaples = 0

is_broken = False
base_dir = '../../data/methods2test_data/train'
# Iterate over all subdirectories (examples) in the base directory
for example_dir in os.listdir(base_dir):
    example_path = os.path.join(base_dir, example_dir)
    
    if os.path.isdir(example_path):

        json_files = glob(os.path.join(example_path, '*.json'))
        
        for json_file in json_files:

            with open(json_file, 'r') as f:
                json_data = json.load(f)
            
            data.append(json_data)

            total_exmaples+=1

            if(total_exmaples>=100):
                print("breaking")
                is_broken = True
                break
    if(is_broken):
        break

df = pd.DataFrame(data)


breaking


In [16]:
df.iloc[0]['focal_class']

{'identifier': 'SourceCodeWriter',
 'superclass': '',
 'interfaces': '',
 'fields': [{'original_string': 'public static final String TEMPLATE_CLASS = "class.vm";',
   'modifier': 'public static final',
   'type': 'String',
   'declarator': 'TEMPLATE_CLASS = "class.vm"',
   'var_name': 'TEMPLATE_CLASS'},
  {'original_string': 'private static Logger log = LoggerFactory.getLogger(SourceCodeWriter.class);',
   'modifier': 'private static',
   'type': 'Logger',
   'declarator': 'log = LoggerFactory.getLogger(SourceCodeWriter.class)',
   'var_name': 'log'},
  {'original_string': 'private final JModel jm;',
   'modifier': 'private final',
   'type': 'JModel',
   'declarator': 'jm',
   'var_name': 'jm'},
  {'original_string': 'private final String methodnamePrefix;',
   'modifier': 'private final',
   'type': 'String',
   'declarator': 'methodnamePrefix',
   'var_name': 'methodnamePrefix'},
  {'original_string': 'private final Calendar now;',
   'modifier': 'private final',
   'type': 'Calenda

## CSR
* Get generated code into a file
* Attempt to compile the file when formatted correctly
* 1 for successful compilation, 0 for unsuccessful

In [ ]:
import subprocess
import os
import csv

def compile_java(java_code, class_name="Main"):
    """
    Writes Java code to a file, compiles it, and returns whether compilation was successful.
    """
    java_filename = f"{class_name}.java"

    with open(java_filename, "w") as java_file:
        java_file.write(java_code)

    compile_process = subprocess.run(["javac", java_filename], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    if compile_process.returncode == 0:
        return True
    else:
        return False

def process_java_files(directory_path, output_csv="results.csv"):
    """
    Iterates through 0.java to 163.java in the given directory,
    compiles each file, and writes the results (1 = success, 0 = failure)
    to a CSV file.
    """
    results = []

    for i in range(164):
        java_file_path = os.path.join(directory_path, f"{i}.java")
        try:
            with open(java_file_path, 'r') as file:
                java_code = file.read()
                success = compile_java(java_code, class_name="Main")
                results.append((i, 1 if success else 0))
        except FileNotFoundError:
            print(f"File not found: {java_file_path}")
            results.append((i, 0))  # or skip if preferred

    # Write to CSV
    with open(output_csv, mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Index', 'CompileSuccess'])
        writer.writerows(results)

# Example usage:
# process_java_files("/path/to/java/files")

paths = ["/media/mujtaba/DATA/nick/UnitTestExamples/data/results/starcoder2-3b__few_shot_first_prompts/java_files/trimmed",
         "/media/mujtaba/DATA/nick/UnitTestExamples/data/results/starcoder2-3b__zero_shot_first_prompts/java_files",
         "/media/mujtaba/DATA/nick/UnitTestExamples/data/results/CodeLlama-7b-Instruct-hf__few_shot_first_prompts/java_files/trimmed",
         "/media/mujtaba/DATA/nick/UnitTestExamples/data/results/CodeLlama-7b-Instruct-hf__zero_shot_first_prompts/java_files"
         ]

for path in paths:
    if "few_shot" in path:
        set_up = path.split('/')[-3]
    else:
        set_up = path.split('/')[-2]
    
    output_csv = set_up + "_CSR.csv"

    process_java_files(path, output_csv=output_csv)


## RSR
* Compile the code
* Run the code
* Return whether the code ran successfully

In [1]:
import subprocess
import os
import csv
import glob

def run_java_code(java_code, main_class_name="Main"):
    """
    Writes Java code to a file, compiles it, and runs the specified main class.
    Cleans up all generated .class files afterward.

    Parameters:
    java_code (str): The Java code as a string.
    main_class_name (str): The name of the class with the main() method to run.

    Returns:
    tuple: (success, output)
    """
    java_filename = f"{main_class_name}.java"

    try:
        # Write Java code to file
        with open(java_filename, "w") as java_file:
            java_file.write(java_code)

        # Compile all classes in the file
        compile_process = subprocess.run(["javac", java_filename],
                                         stdout=subprocess.PIPE,
                                         stderr=subprocess.PIPE,
                                         text=True)

        if compile_process.returncode != 0:
            return False, f"Compilation failed:\n{compile_process.stderr}"

        # Run the main class
        run_process = subprocess.run(["java", main_class_name],
                                     stdout=subprocess.PIPE,
                                     stderr=subprocess.PIPE,
                                     text=True)

        if run_process.returncode == 0:
            return True, run_process.stdout.strip()
        else:
            return False, f"Execution failed:\n{run_process.stderr}"

    finally:
        # Clean up all .class files generated
        for class_file in glob.glob("*.class"):
            try:
                os.remove(class_file)
            except Exception as e:
                print(f"Could not delete {class_file}: {e}")

        # Clean up .java file too
        if os.path.exists(java_filename):
            os.remove(java_filename)

def process_run_success_rate(directory_path, output_csv="run_results.csv"):
    """
    Iterates through 0.java to 163.java in the given directory,
    compiles and runs each, and logs 1 for success and 0 for failure.
    """
    results = []

    for i in range(164):
        java_file_path = os.path.join(directory_path, f"{i}.java")
        try:
            with open(java_file_path, 'r') as file:
                java_code = file.read()
                success, _ = run_java_code(java_code, main_class_name="Main")
                results.append((i, 1 if success else 0))
        except FileNotFoundError:
            print(f"File not found: {java_file_path}")
            results.append((i, 0))

    with open(output_csv, mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Index', 'RunSuccess'])
        writer.writerows(results)

# Example usage:
# process_run_success_rate("/path/to/java/files")

paths = ["/media/mujtaba/DATA/nick/UnitTestExamples/data/results/starcoder2-3b__few_shot_first_prompts/java_files/trimmed",
         "/media/mujtaba/DATA/nick/UnitTestExamples/data/results/starcoder2-3b__zero_shot_first_prompts/java_files",
         "/media/mujtaba/DATA/nick/UnitTestExamples/data/results/CodeLlama-7b-Instruct-hf__few_shot_first_prompts/java_files/trimmed",
         "/media/mujtaba/DATA/nick/UnitTestExamples/data/results/CodeLlama-7b-Instruct-hf__zero_shot_first_prompts/java_files"
         ]

for path in paths:
    if "few_shot" in path:
        set_up = path.split('/')[-3]
    else:
        set_up = path.split('/')[-2]
    
    output_csv = set_up + "_RSR.csv"

    process_run_success_rate(path, output_csv=output_csv)

## CodeBLEU
* Different considerations rather than simple n-gram matching (BLEU)
    * Weighted N-Gram Matching
    * Syntactic AST Match
    * Semantic Data-flow Match

In [13]:
import os
import subprocess

result = subprocess.run(["python3", "../../CodeXGLUE/Code-Code/code-to-code-trans/evaluator/evaluator.py", "-ref", "../../CodeXGLUE/Code-Code/code-to-code-trans/evaluator/references.txt", "-pre", "../../CodeXGLUE/Code-Code/code-to-code-trans/evaluator/predictions.txt"])
print(type(result))
result

BLEU: 61.08 ; Acc: 50.0
<class 'subprocess.CompletedProcess'>


CompletedProcess(args=['python3', '../../CodeXGLUE/Code-Code/code-to-code-trans/evaluator/evaluator.py', '-ref', '../../CodeXGLUE/Code-Code/code-to-code-trans/evaluator/references.txt', '-pre', '../../CodeXGLUE/Code-Code/code-to-code-trans/evaluator/predictions.txt'], returncode=0)

In [ ]:
import subprocess
import os
import csv

def compute_bleu_from_java_files(pred_dir, ref_dir, evaluator_script, output_csv="codebleu_results.csv"):
    """
    Computes CodeBLEU for each prediction .java file compared to its corresponding reference .java file.
    Writes per-example CodeBLEU scores to a CSV.
    """
    results = []

    # Temporary directory to hold single-pair input files
    tmp_dir = "tmp_codebleu_eval"
    os.makedirs(tmp_dir, exist_ok=True)

    for i in range(164):
        pred_path = os.path.join(pred_dir, f"{i}.java")
        ref_path = os.path.join(ref_dir, f"{i}.java")

        if not os.path.exists(pred_path) or not os.path.exists(ref_path):
            # print(f"Missing file for index {i}, skipping {pred_path} and {ref_path}")
            # print(f"Results of check: pred_path -- {os.path.exists(pred_path)}\t ref_path -- {os.path.exists(ref_path)}")
            results.append((i, None))
            continue

        with open(pred_path, "r") as pred_file:
            pred_code = pred_file.read().strip()
        with open(ref_path, "r") as ref_file:
            ref_code = ref_file.read().strip()

        tmp_pred = os.path.join(tmp_dir, "pred.txt")
        tmp_ref = os.path.join(tmp_dir, "ref.txt")

        with open(tmp_pred, "w") as p:
            p.write(pred_code + "\n")
        with open(tmp_ref, "w") as r:
            r.write(ref_code + "\n")

        # Run evaluator.py
        result = subprocess.run(
            ["python3", evaluator_script, "-ref", tmp_ref, "-pre", tmp_pred],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )

        # Parse BLEU score from output
        output = result.stdout.strip()
        print(f"Example {i}: {output}")
        try:
            bleu_line = output.split(";")[0]  # "BLEU: 61.08"
            bleu_score = float(bleu_line.split(":")[1].strip())
        except Exception as e:
            print(f"Error parsing BLEU for index {i}: {output}")
            bleu_score = None

        results.append((i, bleu_score))

    # Clean up temp files
    for f in os.listdir(tmp_dir):
        os.remove(os.path.join(tmp_dir, f))
    os.rmdir(tmp_dir)

    # Write results to CSV
    with open(output_csv, mode="w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["Index", "BLEU"])
        writer.writerows(results)

    print(f"Saved CodeBLEU results to {output_csv}")

# Example usage:
# compute_codebleu_for_each_example(
#     ref_path="../../CodeXGLUE/Code-Code/code-to-code-trans/evaluator/references.txt",
#     pred_path="../../CodeXGLUE/Code-Code/code-to-code-trans/evaluator/predictions.txt",
#     evaluator_script="../../CodeXGLUE/Code-Code/code-to-code-trans/evaluator/evaluator.py"
# )

pred_paths = ["/media/mujtaba/DATA/nick/UnitTestExamples/data/results/starcoder2-3b__few_shot_first_prompts/java_files/trimmed/generations",
         "/media/mujtaba/DATA/nick/UnitTestExamples/data/results/starcoder2-3b__zero_shot_first_prompts/java_files/generations",
         "/media/mujtaba/DATA/nick/UnitTestExamples/data/results/CodeLlama-7b-Instruct-hf__few_shot_first_prompts/java_files/trimmed/generations",
         "/media/mujtaba/DATA/nick/UnitTestExamples/data/results/CodeLlama-7b-Instruct-hf__zero_shot_first_prompts/java_files/generations"
         ]

ref_path = "/media/mujtaba/DATA/nick/UnitTestExamples/UnitTestGenEvaluation/notebooks/organizing_data/targets/Java"

evaluator_path = "/media/mujtaba/DATA/nick/UnitTestExamples/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/evaluator.py"

for path in pred_paths:
    if "few_shot" in path:
        set_up = path.split('/')[-4]
    else:
        set_up = path.split('/')[-3]
    
    output_csv = set_up + "_bleu.csv"

    compute_bleu_from_java_files(path, ref_path, evaluator_path, output_csv=output_csv)


Example 0: 
Error parsing BLEU for index 0: 
Example 1: 
Error parsing BLEU for index 1: 
Example 2: BLEU: 75.82 ; Acc: 76.92
Example 3: 
Error parsing BLEU for index 3: 
Example 4: BLEU: 46.76 ; Acc: 76.92
Example 5: 
Error parsing BLEU for index 5: 
Example 6: 
Error parsing BLEU for index 6: 
Example 7: 
Error parsing BLEU for index 7: 
Example 8: 
Error parsing BLEU for index 8: 
Example 9: BLEU: 22.94 ; Acc: 28.57
Example 10: 
Error parsing BLEU for index 10: 
Example 11: BLEU: 70.77 ; Acc: 76.92
Example 12: 
Error parsing BLEU for index 12: 
Example 13: 
Error parsing BLEU for index 13: 
Example 14: BLEU: 79.72 ; Acc: 76.92
Example 15: BLEU: 62.53 ; Acc: 76.92
Example 16: 
Error parsing BLEU for index 16: 
Example 17: 
Error parsing BLEU for index 17: 
Example 18: 
Error parsing BLEU for index 18: 
Example 19: 
Error parsing BLEU for index 19: 
Example 20: 
Error parsing BLEU for index 20: 
Example 21: 
Error parsing BLEU for index 21: 
Example 22: BLEU: 31.3 ; Acc: 76.92
Example

In [6]:
import os
import csv
from codebleu import calc_codebleu
import traceback

def compute_codebleu_from_java_files(pred_dir, ref_dir, output_csv="codebleu_results.csv", lang="java"):
    """
    Computes CodeBLEU for each prediction .java file compared to its corresponding reference .java file.
    Uses codebleu.calc_codebleu() and writes per-example scores to a CSV.
    """
    results = []

    for i in range(164):
        pred_path = os.path.join(pred_dir, f"{i}.java")
        ref_path = os.path.join(ref_dir, f"{i}.java")

        if not os.path.exists(pred_path) or not os.path.exists(ref_path):
            print(f"Missing file for index {i}, skipping.")
            results.append((i, None))
            continue

        with open(pred_path, "r", encoding="utf-8") as pred_file:
            prediction = pred_file.read().strip()

        with open(ref_path, "r", encoding="utf-8") as ref_file:
            reference = ref_file.read().strip()

        try:
            result = calc_codebleu(
                [reference],
                [prediction],
                lang=lang,
                weights=(0.25, 0.25, 0.25, 0.25),  # default CodeBLEU weights
                tokenizer=None
            )
            codebleu_score = result["codebleu"]
        except Exception as e:
            print(f"Error computing CodeBLEU for index {i}: {e}")
            traceback_str = traceback.format_exc()
            print(traceback_str)
            codebleu_score = None

        results.append((i, codebleu_score))

    # Write results to CSV
    with open(output_csv, mode="w", newline="", encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["Index", "CodeBLEU"])
        writer.writerows(results)

    print(f"Saved CodeBLEU results to {output_csv}")

# Example usage:
# compute_codebleu_from_java_files(
#     pred_dir="path/to/predictions",
#     ref_dir="path/to/references"
# )

pred_paths = ["/media/mujtaba/DATA/nick/UnitTestExamples/data/results/starcoder2-3b__few_shot_first_prompts/java_files/trimmed/generations",
         "/media/mujtaba/DATA/nick/UnitTestExamples/data/results/starcoder2-3b__zero_shot_first_prompts/java_files/generations",
         "/media/mujtaba/DATA/nick/UnitTestExamples/data/results/CodeLlama-7b-Instruct-hf__few_shot_first_prompts/java_files/trimmed/generations",
         "/media/mujtaba/DATA/nick/UnitTestExamples/data/results/CodeLlama-7b-Instruct-hf__zero_shot_first_prompts/java_files/generations"
         ]

ref_path = "/media/mujtaba/DATA/nick/UnitTestExamples/UnitTestGenEvaluation/notebooks/organizing_data/targets/Java"

for path in pred_paths:
    if "few_shot" in path:
        set_up = path.split('/')[-4]
    else:
        set_up = path.split('/')[-3]
    
    output_csv = set_up + "_bleu.csv"

    compute_codebleu_from_java_files(path, ref_path, output_csv=output_csv)

Error computing CodeBLEU for index 0: Tree-sitter language for java not available. Please install the language parser using `pip install tree-sitter-java`.
Traceback (most recent call last):
  File "/media/mujtaba/DATA/nick/miniconda3/envs/codebleu/lib/python3.9/site-packages/codebleu/utils.py", line 134, in get_tree_sitter_language
    import tree_sitter_java
ModuleNotFoundError: No module named 'tree_sitter_java'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_3570378/448273514.py", line 29, in compute_codebleu_from_java_files
    result = calc_codebleu(
  File "/media/mujtaba/DATA/nick/miniconda3/envs/codebleu/lib/python3.9/site-packages/codebleu/codebleu.py", line 41, in calc_codebleu
    tree_sitter_language = get_tree_sitter_language(lang)
  File "/media/mujtaba/DATA/nick/miniconda3/envs/codebleu/lib/python3.9/site-packages/codebleu/utils.py", line 179, in get_tree_sitter_language
    raise ImportErro

## EM (Exact Match)
* Purely Matching whether all characters match
* Key things to think about:
    * Removing white space
    * Cleaning output for direct comparison
    * Simple string comparison works perfectly

In [3]:
def exact_match(prediction, reference):
    prediction = prediction.strip()
    reference = reference.strip()

    # OTHER CLEANING GOES HERE FOR EACH OUTPUT

    return prediction == reference

output = "def main():\n\tprint('Hello World')"
target = "def main():\n\tprint('Hello World')"

exact_match(output, target)

True

## Acc@10 (Any top 10 generations exactly match)
* Model needs to generation 10 responses for each example
* Iterate through all 10 example for comparison
* Use same logic as EM

In [ ]:
def acc_at_10(predictions, reference):
    for prediction in predictions:
        prediction = prediction.strip()
        print(prediction)

    reference = reference.strip()

    # OTHER CLEANING GOES HERE FOR EACH OUTPUT

    for prediction in predictions:
        if prediction==reference:
            return True
    return False

predictions = ["def hello_world:\n\tprint('Hello, World!')", "def hello_world:\n\tprint('Hello, World)", "def hello_world:\n\tprint('Hello, World!\")", "def hello_world:\n\tprint('Hello, World!)", "def hello_world:\n\tprint('Hello, World!)", "def hello_world:\n\tprint(\"Hello, World!\")", "def print_hello_world:\n\tprint('Hello, World!)", "def helloworld:\n\tprint('Hello, World!)", "def hello_world:\n\tprint('Hello, World)", "def hello_world:\n\tprint('Hello World!)"] 
reference = "def hello_world:\n\tprint('Hello, World!')"   

## Edit Similarity (Levenshtein Edit Distance)

In [9]:
import Levenshtein

def levenshtein_distance(str1, str2):
    return Levenshtein.distance(str1, str2)

# Example usage
str1 = "kitten"
str2 = "sitting"
distance = levenshtein_distance(str1, str2)
print(f"The Levenshtein distance between '{str1}' and '{str2}' is {distance}.")
print(1-distance)

The Levenshtein distance between 'kitten' and 'sitting' is 3.
-2


## Line Coverage
* Lines Executed / Total Executable Lines

## Branch Coverage
* Branches Executed / Total Executable Lines